In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"skylarpro","key":"61c6a5a697295af30a778f1880685730"}'}

In [ ]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

rm: cannot remove '/root/.kaggle': No such file or directory
ref                                                          title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
victorsoeiro/netflix-tv-shows-and-movies                     Netflix TV Shows and Movies                           2MB  2022-05-15 00:01:23           7624        236  1.0              
devansodariya/student-performance-data                       Student Performance Dataset                           7KB  2022-05-26 13:55:09           3329        131  0.9705882        
iamsouravbanerjee/software-professional-salaries-2022        Salary Dataset - 2022                               526KB  2022-06-10 06:02:59           1236         37  1.0              
paradisejoy/to

In [ ]:
https://www.kaggle.com/datasets/arkhoshghalb/twitter-sentiment-analysis-hatred-speech

!kaggle datasets download "gpreda/covid19-tweets"
!unzip /content/covid19-tweets.zip

In [ ]:
!kaggle datasets download "arkhoshghalb/twitter-sentiment-analysis-hatred-speech"
# !unzip /content/arkhoshghalb/twitter-sentiment-analysis-hatred-speech.zip
!unzip /content/twitter-sentiment-analysis-hatred-speech.zip

  0% 0.00/1.89M [00:00<?, ?B/s]
100% 1.89M/1.89M [00:00<00:00, 199MB/s]


##Подготовьте набор данных для дальнейшей работы

Для данной задачи подготовка заключается в удалении лишних символов и знаков препинания, а также приведении к нижнему регистру и токенизации

In [ ]:
import re
import pandas as pd
import string
import numpy as np

In [ ]:
def _processing_text(proc_text):
        proc_text = proc_text.lower()
        proc_text = proc_text.replace("\n"," ")
        proc_text = re.sub(r"http\S+", "",proc_text)
        proc_text = re.sub("#(\w+)", " ", proc_text)
        proc_text = re.sub("@(\w+)", " ", proc_text)
        
        proc_text = re.sub('[^a-z]', " ", proc_text)
        proc_text = re.sub(" +", " ", proc_text)
        
      
        proc_text = [char for char in proc_text if char not in string.punctuation]
        proc_text = ''.join(proc_text)
        return proc_text

In [ ]:
df_2["text"].iloc[2]

NameError: ignored

In [ ]:
df_2["text_proc"].iloc[2]

In [ ]:
df_2 = pd.read_csv("/content/covid19_tweets.csv")
df_2["text_proc"] = df_2["text"].apply(_processing_text)
df_2["text_proc"]

0         if i smelled the scent of hand sanitizers toda...
1         hey and wouldn t it have made more sense to ha...
2          trump never once claimed was a hoax we all cl...
3          the one gift has give me is an appreciation f...
4                             july media bulletin on novel 
                                ...                        
179103    thanks for nominating me for the challenge i n...
179104                            the year of insanity lol 
179105     a powerful painting by juan lucena it s a tri...
179106    more than students test positive for at major ...
179107                            i stop when i see a stop 
Name: text_proc, Length: 179108, dtype: object

##Выделите уникальные слова, создайте словарь, в котором ключами будут являться слова, а значениями соответствующие им индексы

Используйте метод fit_on_texts  модуля keras.preprocessing.text. Должен получиться один словарь, содержащий слова со всех строк набора данных. В реализации этого и дальнейших пунктов могут помочь полезные материалы.

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
training_corpus = df_2["text_proc"].to_numpy()

In [ ]:
from transformers import AutoTokenizer
old_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
old_res = old_tokenizer.tokenize(training_corpus[0])

In [ ]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 30000)

In [ ]:
new_res = tokenizer.tokenize(training_corpus[0])

In [ ]:
assert not np.all(old_res == new_res)

##Создайте модель нейронной сети

Из библиотеки keras  импортируйте тип модели Sequential, добавьте в неё 3 слоя LSTM и слой Dense. Так как мы имеем дело с многоклассовой классификацией (количество классов соответствует количеству слов в наборе данных), в качестве функции потери используется categorical_crossentropy. Вы можете попробовать улучить модель путём подбора гиперпараметров.

1. Сделать LSTM
2. Добавить Conv в cделанную LSTM
3. Добавить Attention в LSTM на шаге два
4. Добавить Bidirectional


In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F
from collections import OrderedDict
import math

In [ ]:
class LSTMLayerEncoder(nn.Module):
  def __init__(self,input_size, hidden_size):
    super().__init__()
    self.hidden_size = hidden_size

    self.W = nn.Parameter(torch.Tensor(input_size, hidden_size * 4))
    self.V = nn.Parameter(torch.Tensor(hidden_size,hidden_size * 4))
    self.bias = nn.Parameter(torch.Tensor(hidden_size * 4))
    self.init_weights()
    self.dorpout = nn.Dropout(p=0.1)

  def init_weights(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)
    
  def forward(self,input, init_states=None):
    bs, seq_len, _ = input.shape

    if init_states is None:
      h_t = torch.zeros((bs, self.hidden_size)).cuda()
      c_t = torch.zeros((bs, self.hidden_size)).cuda()
    else:
      h_t, c_t = init_states

    HS = self.hidden_size
    hidden_seq = []

    for t in range(seq_len):
      x_t = input[:,t,:]
      gates = x_t@self.W + h_t@self.V + self.bias

      # (b_s,inp_size)@ (inp_size, hid*4) = (b_s, hid*4) 
      # (b_s, hid) @ (hid,hid*4) = (b_s, hid*4) 
      # (b_s, hid*4) + (hid*4) = (b_s, hid*4)
      i_t, f_t, g_t, o_t = (
                F.sigmoid(gates[:, :HS]), # input
                F.sigmoid(gates[:, HS:HS*2]), # forget
                F.tanh(gates[:, HS*2:HS*3]),
                F.sigmoid(gates[:, HS*3:]), # output
      )
      
      c_t = c_t*f_t + i_t*g_t
      h_t = o_t * F.tanh(c_t)

      h_t = self.dorpout(h_t)

      hidden_seq.append(h_t.unsqueeze(0))

    hidden_seq = torch.cat(hidden_seq, dim=0) 
    hidden_seq = hidden_seq.transpose(0, 1).contiguous()
    return hidden_seq, (h_t, c_t)

class LSTMLayerDecoder(nn.Module):
  def __init__(self,input_size, hidden_size, max_len_generation, vocab_size, embeding_model):
    super().__init__()
    self.hidden_size = hidden_size
    self.max_len_generation = max_len_generation

    self.W = nn.Parameter(torch.Tensor(input_size, hidden_size * 4))
    self.V = nn.Parameter(torch.Tensor(hidden_size,hidden_size * 4))
    self.bias = nn.Parameter(torch.Tensor(hidden_size * 4))
    self.init_weights()

    self.embeding_model = embeding_model
    self.dorpout = nn.Dropout()

    self.projection = nn.Sequential(
        nn.Linear(hidden_size, vocab_size),
        nn.Sigmoid()
        )

  def init_weights(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)
    
  def forward(self,input, init_states):

    h_t, c_t = init_states

    HS = self.hidden_size
    predict_words = []
    hash_prediction = []


    x_t = input[:,0,:]
    for _ in range(self.max_len_generation):
      gates = x_t@self.W + h_t@self.V + self.bias

      i_t, f_t, g_t, o_t = (
                F.sigmoid(gates[:, :HS]), # input
                F.sigmoid(gates[:, HS:HS*2]), # forget
                F.tanh(gates[:, HS*2:HS*3]),
                F.sigmoid(gates[:, HS*3:]), # output
      )
      
      c_t = c_t*f_t + i_t*g_t
      h_t = o_t * F.tanh(c_t)

      predict_word = self.projection(h_t)
      next_pred = torch.argmax(predict_word, dim = 1).unsqueeze(1)
      x_t = self.embeding_model(next_pred)[:,0,:]

      hash_prediction.append(predict_word.unsqueeze(1)) #(b_s, vocab_size,1)

      predict_words.append(next_pred) # (b_s, 1)

    predict_words = torch.cat(predict_words, dim=1) 

    hash_prediction = torch.cat(hash_prediction, dim=1)  #(b_s, vocab_size, len_generation)

    return hash_prediction, predict_words

class CustomLSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers,
               max_len_generation, vocab_size, embeding_model):
    super().__init__()
    self.LSTM_encoder1 = LSTMLayerEncoder(input_size, hidden_size) #nn.Sequential(OrderedDict({f"lstm_layer{i+1}":
     #for i in range(num_layers)}))
    self.LSTM_encoder2 = LSTMLayerEncoder(hidden_size, hidden_size)
    self.dorpout = nn.Dropout(p=0.15)
    self.LSTM_encoder3 = LSTMLayerEncoder(hidden_size, hidden_size)

    self.LSTM_decoder = LSTMLayerDecoder(input_size, hidden_size, max_len_generation,
                                         vocab_size,embeding_model) #nn.Sequential(OrderedDict({f"lstm_layer{i+1}":
     #for i in range(num_layers)}))

  def forward(self, input_encoder, input_decoder, init_states = None):
    hidden_seq, _ = self.LSTM_encoder1(input_encoder)
    hidden_seq = self.dorpout(hidden_seq)
    hidden_seq, _ = self.LSTM_encoder2(hidden_seq)
    hidden_seq = self.dorpout(hidden_seq)
    _, (h_t, c_t) = self.LSTM_encoder3(hidden_seq)
    out = self.LSTM_decoder(input_decoder, (h_t, c_t))
    return out

### Dataloader 


In [ ]:
from transformers import BertTokenizer
from transformers import BertModel

BERT_MODEL = "distilbert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL) 
model_bert = BertModel.from_pretrained(BERT_MODEL)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing BertModel: ['distilbert.transformer.layer.3.attention.v_lin.weight', 'distilbert.transformer.layer.4.output_layer_norm.bias', 'distilbert.transformer.layer.4.ffn.lin1.bias', 'distilbert.transformer.layer.0.output_layer_norm.weight', 'distilbert.transformer.layer.2.ffn.lin2.weight', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.4.sa_layer_norm.bias', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilb

In [ ]:
from torch.utils.data import Dataset, DataLoader,SubsetRandomSampler
import torch

class TweetDataset(Dataset):

  def __init__(self,data, tokenizer, max_seq_length):
    self.text_data = data
    self.tokenizer = tokenizer
    self.max_seq_length = max_seq_length

  def __getitem__(self,idx):
        text = self.text_data[idx]
        input_ids = self.tokenizer.encode(f"{text}")[1:]
        

        if len(input_ids) > self.max_seq_length:
            input_ids = input_ids[:self.max_seq_length]

        # Zero-pad up to the sequence length.
        padding = [0] * (self.max_seq_length - len(input_ids))
        input_ids += padding

        attention_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (self.max_seq_length - len(input_ids))
        input_ids += padding
        attention_mask += padding

        decoder_start = [101]

        return torch.tensor(input_ids), torch.tensor(attention_mask), torch.tensor(decoder_start)

  def __len__(self,):
    return self.text_data.shape[0]

LEN_SEQ = 7
train_dataset = TweetDataset(training_corpus, tokenizer, max_seq_length = LEN_SEQ)

In [ ]:
training_corpus

array(['if i smelled the scent of hand sanitizers today on someone in the past i would think they were so intoxicated that ',
       'hey and wouldn t it have made more sense to have the players pay their respects to the a ',
       ' trump never once claimed was a hoax we all claim that this effort to ',
       ...,
       ' a powerful painting by juan lucena it s a tribute to the grandparents who died of covid and the grandc ',
       'more than students test positive for at major university abc news ',
       'i stop when i see a stop '], dtype=object)

In [ ]:
def sempler_eval(data_train, batch_size = 128,):
    
    data_size = len(data_train)

    eval_indices = np.array(list(range(data_size)))
    np.random.shuffle(eval_indices)

    eval_sampler = SubsetRandomSampler(eval_indices)

    eval_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size,
                                              sampler=eval_sampler,)

    return eval_loader

train_loader = sempler_eval(train_dataset)

In [ ]:
vocab_size = tokenizer.vocab_size
embeding_model = model_bert.embeddings
embeding_model.cuda()
embeding_model.requires_grad = False
embeding_model.eval()
model = CustomLSTM(768, 2048, 1, LEN_SEQ, vocab_size, embeding_model)
model.cuda()

CustomLSTM(
  (LSTM_encoder1): LSTMLayerEncoder(
    (dorpout): Dropout(p=0.1, inplace=False)
  )
  (LSTM_encoder2): LSTMLayerEncoder(
    (dorpout): Dropout(p=0.1, inplace=False)
  )
  (dorpout): Dropout(p=0.15, inplace=False)
  (LSTM_encoder3): LSTMLayerEncoder(
    (dorpout): Dropout(p=0.1, inplace=False)
  )
  (LSTM_decoder): LSTMLayerDecoder(
    (embeding_model): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (dorpout): Dropout(p=0.5, inplace=False)
    (projection): Sequential(
      (0): Linear(in_features=5096, out_features=30522, bias=True)
      (1): Sigmoid()
    )
  )
)

In [ ]:
from tqdm import tqdm 
from torch.optim import Adam

In [ ]:
model.train()
loss = nn.CrossEntropyLoss()
for name,param in model.named_parameters():
  if name.find("embeding_model")!=-1:
    param.requires_grad = False

optim = Adam([param for param in model.parameters() if param.requires_grad ],lr = 1e-02, weight_decay=1e-4)

In [ ]:
epochs = 1
loss_array = []
Acc_array = []
for epoch in range(epochs):
  loss_values = 0
  for idx, (text_to_encoder, attentiom_mask, text_to_decoder) in tqdm(enumerate(train_loader)):

    text_to_encoder_cuda = text_to_encoder.cuda()
    attentiom_mask_cuda = attentiom_mask.cuda()
    text_to_decoder_cuda = text_to_decoder.cuda()

    input_to_LSTM_encoder = embeding_model(text_to_encoder_cuda)
    input_to_LSTM_decoder = embeding_model(text_to_decoder_cuda)


    pred_probably, _ =  model(input_to_LSTM_encoder, input_to_LSTM_decoder)
    del input_to_LSTM_encoder
    del input_to_LSTM_decoder
    del attentiom_mask_cuda
    del text_to_decoder_cuda

    loss_value = loss(pred_probably.transpose(1,2), text_to_encoder_cuda)
    optim.zero_grad()
    loss_value.backward()
    optim.step()
    loss_values+=loss_value

    if idx!=0:
      print(f"loss {loss_values/idx} on {idx} iter")
    
    del pred_probably
    del text_to_encoder_cuda
    
  loss_array.append(loss_values)
  print(f"Epoch {epoch}")
  print(f"Loss {loss_values/idx+1}")

In [ ]:
def generation_sentence(model_gen, tokenizer):
  model_gen.eval()
  c_t = 2022*torch.rand((5,2048)).cuda()
  h_t = 502*torch.rand((5,2048)).cuda()
  token_start = embeding_model(torch.tensor([101]).unsqueeze(0).cuda())
  _,result = model_gen.LSTM_decoder(token_start,(h_t, c_t))
  decode_text = []
  for sample in result:
      decode_text.append(tokenizer.decode(sample))
  del c_t
  del h_t
  del token_start
  return decode_text

In [ ]:
generation_sentence(model, tokenizer)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1933: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


['ru b b b b',
 'school b b b b',
 'which b b b b',
 'could b b b b',
 'ru b b b b']

In [ ]:
from typing import List

def generation_sentence_from_title(model_gen, model_emb, text_input: List[str], tokenizer):
  prep_text = list(map(_processing_text,text_input))
  tokenize_text = list(map(tokenizer.encode, prep_text))
  model_gen.eval()
  model_emb.eval()
  def form_vectors(tokenize_text):
    res_input = []
    res_att_mask = []

    for text in tokenize_text:
      text = text[1:]
      if len(text) > LEN_SEQ:
            text = text[:LEN_SEQ]

      padding = [0] * (LEN_SEQ - len(text))
      text += padding

      attention_mask = [1] * len(text)

      padding = [0] * (LEN_SEQ - len(text))
      text += padding
      attention_mask += padding

      res_input.append(text)
      res_att_mask.append(attention_mask)
    res_input = torch.tensor(res_input)
    res_att_mask = torch.tensor(res_att_mask)
    decoder_start = torch.tensor([101]*len(tokenize_text)).unsqueeze(1)

    return res_input, res_att_mask, decoder_start

  res_input, res_att_mask, decoder_start = form_vectors(tokenize_text)

  input_to_LSTM_decoder = model_emb(decoder_start.cuda())
  input_to_LSTM_encoder = model_emb(res_input.cuda())

  _, result =  model_gen(input_to_LSTM_encoder, input_to_LSTM_decoder)
  decode_text = []
  for sample in result:
      decode_text.append(tokenizer.decode(sample))

  del input_to_LSTM_decoder
  del input_to_LSTM_encoder
  del result
  return decode_text

In [ ]:
generation_sentence_from_title(model,embeding_model,["covid19 india china asd sad"],tokenizer)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1933: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


['b b b b b']

Проблема переобучения на b b b b возможна в виду плохих данных непригодных для генерации.
Из примера выше видно что если мы подадим сильный шум то сетка что-то предскажет не с b b но после она все равно скатется к этим значениям. Значит она переобучилась(